Download the data.

In [1]:
import wget

data_url = "http://human.brain-map.org/api/v2/well_known_file_download/157721937"
print('Beginning file download with wget module')
wget.download(data_url, 'T1.nii.gz')

Beginning file download with wget module
100% [..........................................................................] 4476196 / 4476196

'T1.nii.gz'

Read the nifti file using nibabel.

In [7]:
import nibabel as nib
img = nib.load('T1.nii.gz')
img_data = img.get_fdata()
img_data.shape

(182, 182, 218)

The data is formatted as a big numpy array. Find the coordinates of the data using nibabel functions, and concatenate with the contrast to get a data frame.

In [29]:
import numpy as np
from nibabel.affines import apply_affine
import pandas as pd

xi, yi, zi = np.meshgrid(np.arange(img_data.shape[0]), np.arange(img_data.shape[1]), np.arange(img_data.shape[2]))
coords_transformed = apply_affine(img.affine, np.stack((xi.ravel(), yi.ravel(), zi.ravel()), axis=1))
data_table = np.concatenate((coords_transformed, img_data.ravel().reshape(-1, 1)), 1)

df = pd.DataFrame(data_table, columns=['position_x', 'position_y', 'position_z', 'contrast'])

In [26]:
df.query('contrast > 0').head()

,position.x,position.y,position.z,contrast
278016,90.0,42.0,83.0,2.0
278023,90.0,35.0,83.0,1.0
278234,89.0,42.0,83.0,9.0
278235,89.0,41.0,83.0,2.0
278240,89.0,36.0,83.0,1.0


Now we have a data table with the three first columns representing x, y, and z, and the fourth the contrast. Let's save this as a pcache file. The data is pretty big so let's downsample 2X in each dimension to start with.

In [32]:
def df_to_pcache(df):
    """Take a pandas dataframe and dumps it as a pcache file.
    See this repo: https://github.com/peeweek/pcache
    
    Args:
        df = a dataframe
    
    Returns:
        A string
    """
    header = b"pcache\n"
    header += b"comment PCACHE file Exported from Numpy\n"
    header += b"format binary 1.0\n"
    header += b"elements %d\n" % df.shape[0]
    
    # Out of laziness, I only implemented one data type per table.
    df_ = df.to_numpy()
    for n in df.columns:
        if df[n].dtype == np.float32:
            header += b"property float " + bytes(n, 'ascii') + b"\n"
        elif df[n].dtype == np.uint16:
            header += b"property ushort " + bytes(n, 'ascii') + b"\n"
        else:
            raise NotImplementedError("dtype %s not implemented" % coords.dtype)
    header += b"end_header\n"
    
    return header + np.ascontiguousarray(df_).tostring()

df_sub = df.query('(contrast > 0) and (position_x % 2 == 0) and (position_y % 2 == 0) and (position_z % 2 == 0)')
pcache_str = df_to_pcache(df_sub.astype(np.float32))

# Now save to disk
with open('T1.pcache', 'wb') as f:
    f.write(pcache_str)